# Aprendiendo Django

## Fuentes
https://realpython.com/get-started-with-django-1/  
https://www.youtube.com/watch?v=EEiqGjCNLRs


## Preparar el entorno virtual
Se recomienda trabajar con un entorno virtual, por lo que haríamos un `.venv`:  
```bash
python -m venv .venv --upgrade-deps
.venv/Scripts/activate

pip install Django
```

Aunque también es recomendable crear previamente un `requirements.txt` y hacer un `pip install -r requirements.txt`.

## Empezar un proyecto
ℹ️ Tanto para el nombre de proyecto como para el de las apps he comprobado que existen ciertas limitaciones. La primera es no usar guiones para separar. Mejor todo junto. La segunda, en el caso de las aplicaciones, es que el nombre no debería coincidir con el de un módulo de Python, ya que podría crear conflictos. "portfolio", por ejemplo, no es un nombre válido.  
⚠️ Tampoco se deben cambiar los nombres una vez los creemos, ya que está todo personalizado y pueden salir mal varias cosas.  

Se crea el proyecto con el comando:  
```bash
django-admin startproject <NAMEPROJECT> .
```

" ⚠️ Don’t forget to add the dot (.) at the end of the command above. The dot prevents Django from creating a nested project directory for your portfolio project. Otherwise, you’d end up with a personal_portfolio/ folder that contains a personal_portfolio/ subdirectory.


Comprobamos que todo funciona corriendo el servidor:  
```bash
cd <NAMEPROJECT>
python manage.py runserver
```

Por defecto podremos abrir el servidor en http://127.0.0.1:8000/, pero podemos cambiar el puerto si lo añadimos al comando después de `runserver`.  

### Añadir aplicación
Lo siguiente es crear una app de django. Para seguir, tenemos que parar el server con CNTRL+C y escribimos:  
```bash
python manage.py startapp <NAMEAPP>
```

Después de esto, hay que instalar la app en el proyecto (en algún ejemplo no hacía falta, pero vamos a hacerlo bien, ¿verdad?). En el archivo `settings.py` del proyecto, la añadimos a la constante INSTALLED_APPS:

In [ ]:
# personal_portfolio/settings.py

# ...

INSTALLED_APPS = [
    "nameapp.apps.NameappConfig",  # según el nombre de la app, claro
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

# ...

En efecto, en la carpeta de la app ("nameapp" o la que sea) tenemos el archivo apps.py y, dentro, la clase `NameappConfig` (o el nombre que tenga).  

## Crear una Vista
Dentro del directorio de la app, abrimos `views.py` y añadiremos la petición para una página HTML que podemos crear antes o después:

In [ ]:
# <NAMEAPP>/views.py

from django.shortcuts import render


# Create your views here.
def home(request):
    return render(request, "nameapp/home.html", {})

Primero va un `HttpRequest`, luego un `template_name`. La llave de diccionario al final es el `context:Mapping`. Usa como parámetros variables que establecemos por ejemplo al buscar dentro de una db y las pasamos para que la plantilla html pueda renderizarlos como etiquetas.



### Añadir plantillas
Ahora creamos una carpeta llamada `templates` dentro de nuestra app. Se aconseja ponerle después OTRA VEZ el nombre de la app. Por ejemplo, la ruta final sería `nameapp/templates/nameapp`.

Podemos crear una página HTML (en el ejemplo tiene el nombre "home", en otro era "index", etc.) en esta carpeta por cualquier método. Por ejemplo, un `echo loren ipsum > index.html`, que será una página que únicamente contendrá el texto "loren ipsum". En el tutorial de Real Python dice que pongamos lo siguiente:  

```html
<!-- pages/templates/pages/home.html -->

<h1>Hello, World!</h1>
```  

⚠️ Mejor crear este archivo de otra forma o, al menos, asegurarnos de que tiene codificación UTF-8.  


### Añadir una Ruta
En el archivo `urls.py` dentro del directorio del proyecto, vamos a modificar lo siguiente:

In [ ]:
# <NAMEPROJECT>/urls.py

from django.contrib import admin
from django.urls import path


urlpatterns = [

    path("admin/", admin.site.urls),

]

⬇️⬇️⬇️

In [ ]:
# <NAMEPROJECT>/urls.py

from django.contrib import admin
from django.urls import path, include


urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("nameapp.urls")),
]

Pero `nameapp.urls` aún no existe, así que lo creamos y añadimos las rutas. Con Real Python: 

In [ ]:
# pages/urls.py

from django.urls import path
from pages import views

urlpatterns = [
    path("", views.home, name="home"),
]

Con Python Simplified (Marie):

In [ ]:
from django.urls import path
from . import views

urlpatterns = [path("myview/", views.myview)]

## Añadir Bootstrap (opcional)
Antes de empezar, habría que crear la carpeta de plantillas madre, de la que beberán los demás. En la carpeta raíz (no la del proyecto), crear `templates` y el archivo `templates/base.html`. Dentro, añadimos lo siguiente:

```html
<!-- templates/base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>{% block title %}My Personal Portfolio{% endblock title %}</title>
    <link
        href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.1/dist/css/bootstrap.min.css"
        rel="stylesheet"
    >
</head>
<body class="container">
{% block page_content %}{% endblock page_content %}
</body>
</html>
```
Y retocamos el archivo `home.html`:

```html
<!-- pages/templates/pages/home.html -->

{% extends "base.html" %}

{% block page_content %}
    <h1>Hello, World!</h1>
{% endblock page_content %}
```


Para que Django tenga conocimiento de que la página base.html existe, hay que ir a la carpeta del proyecto y entrar en `settings.py` para establecer la dirección en la constante `TEMPLATES`, y dentro de la lista vacía que es el campo de la clave `DIRS`, hacer un join de `BASE_DIR` y la carpeta donde se encuentra templates.  

" Before you can see the inheritance and the new styled application in action, you need to tell your Django example project that base.html exists. The default settings register templates/ directories in each app, but not in the root directory itself. In personal_portfolio/settings.py, update TEMPLATES:

In [ ]:
# personal_portfolio/settings.py

# ...

TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [
            BASE_DIR / "templates/",
        ],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                "django.template.context_processors.debug",
                "django.template.context_processors.request",
                "django.contrib.auth.context_processors.auth",
                "django.contrib.messages.context_processors.messages",
            ]
        },
    }
]

# ...

Gracias al módulo `pathlib`, hace el join solo sin necesidad de aplicar un `Path.joinpath(BASE_DIR, "templates/")`.

# Lista de comandos útiles

```
django-admin startproject <NAMEPROJECT> .

python manage.py runserver (<PORT>)
python manage.py startapp <NAMEAPP>
python manage.py shell
python manage.py makemigrations (<NAMEAPP>) ❓
python manage.py migrate (<NAMEAPP>)
python manage.py createsuperuser

```

# Por hacer
Enlaces (por ejemplo "proyectos" no funciona en lugar de "proyectos/")
Modificar los textos de "proyectos". Están los originales y hay cosas que ya no cuadran. 
Crear un mapeador o algo así. 